In [19]:
import math

# # Parameters
# n_s = 5
# n_d = 10
# v_d = 2 # m/sec
# R_min = 10  # m
# R_c = 200  # m
# R_det = 300  # m
# r = 3
# l = 2
# m_f = 0

# Optional Parameters


# alpha = 6
# S = 10  # sec
# C = 240  # sec

# Drawing Parameters
adsSize = 5
droneSize = 5
health_bar_width = 20
health_bar_height = 6

In [20]:
import pygame
import sys
import random
import math

# Pygame initialization
pygame.init()

# Set up the screen
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Simulation")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
GRAY = (200, 200, 200)

# Fonts
font = pygame.font.SysFont(None, 32)

# Variable names
n_s = None
n_d = None
v_d = None
R_min = None
R_c = None
R_det = None
r = None
l = None
m_f = None
alpha = None
S = None
C = None



# Parameters

parameters = {
    "Number of ADS": "2",
    "Number of Drones": "5",
    "Velocity of Drone": "3",
    "Min Allowed Proximity for Drone": "10",
    "Critical Region": "200",
    "Detection Region": "300",
    "Hits to kill a Drone": "3",
    "Lasers in one ADS": "2",
    "Failure Probability of a strike": "0.25",
    "Max Strikes before ADS requires Cool-down": "6",
    "Strike Time": "10",
    "Cooldown Time": "240"
}

# Input fields
input_fields = {}
active_field = ""

def draw_text_input(x, y, label, value, active):
    text_surface = font.render(label, True, BLACK)
    screen.blit(text_surface, (x, y))
    rect = pygame.Rect(x + 480, y, 100, 32)  # Reduced height to 40 pixels
    if active:
        pygame.draw.rect(screen, RED, rect, 2)
    else:
        pygame.draw.rect(screen, GRAY, rect, 2)
    input_text = font.render(value, True, BLACK)
    screen.blit(input_text, (x + 490, y+5))

def draw_ui():
    screen.fill(WHITE)
    y_offset = 30  # Reduced height to 40 pixels
    for key, value in parameters.items():
        active = key == active_field
        draw_text_input(50, y_offset, f"{key}:", value, active)
        y_offset += 40  # Reduced height to 40 pixels
    pygame.draw.rect(screen, BLACK, (screen_width//2 - 50, screen_height - 100, 100, 50))
    next_button_text = font.render("Next", True, WHITE)
    screen.blit(next_button_text, (screen_width//2 - 30, screen_height - 80))

def update_variables():
    global n_s, n_d, v_d, R_min, R_c, R_det, r, l, m_f, alpha, S, C
    flag = True
    try:
        n_s = int(parameters["Number of ADS"])
    except ValueError:
        flag = False
        print("Invalid input for Number of ADS. Please enter a valid numerical value.")
    try:
        n_d = int(parameters["Number of Drones"])
    except ValueError:
        flag = False
        print("Invalid input for Number of Drones. Please enter a valid numerical value.")
    try:
        v_d = float(parameters["Velocity of Drone"])
    except ValueError:
        flag = False
        print("Invalid input for Velocity of Drone. Please enter a valid numerical value.")
    try:
        R_min = int(parameters["Min Allowed Proximity for Drone"])
    except ValueError:
        flag = False
        print("Invalid input for Min Allowed Proximity for Drone. Please enter a valid numerical value.")
    try:
        R_c = int(parameters["Critical Region"])
    except ValueError:
        flag = False
        print("Invalid input for Critical Region. Please enter a valid numerical value.")
    try:
        R_det = int(parameters["Detection Region"])
    except ValueError:
        flag = False
        print("Invalid input for Detection Region. Please enter a valid numerical value.")
    try:
        r = int(parameters["Hits to kill a Drone"])
    except ValueError:
        flag = False
        print("Invalid input for Hits to kill a Drone. Please enter a valid numerical value.")
    try:
        l = int(parameters["Lasers in one ADS"])
    except ValueError:
        flag = False
        print("Invalid input for Lasers in one ADS. Please enter a valid numerical value.")
    try:
        m_f = float(parameters["Failure Probability of a strike"])
    except ValueError:
        flag = False
        print("Invalid input for Failure Probability of a strike. Please enter a valid numerical value.")
    try:
        alpha = int(parameters["Max Strikes before ADS requires Cool-down"])
    except ValueError:
        flag = False
        print("Invalid input for Max Strikes before ADS requires Cool-down. Please enter a valid numerical value.")
    try:
        S = int(parameters["Strike Time"])
    except ValueError:
        flag = False
        print("Invalid input for Strike Time. Please enter a valid numerical value.")
    try:
        C = int(parameters["Cooldown Time"])
    except ValueError:
        flag = False
        print("Invalid input for Cooldown Time. Please enter a valid numerical value.")
    return flag
    
def print_variables():
    print("n_s:", n_s)
    print("n_d:", n_d)
    print("v_d:", v_d)
    print("R_min:", R_min)
    print("R_c:", R_c)
    print("R_det:", R_det)
    print("r:", r)
    print("l:", l)
    print("m_f:", m_f)
    print("alpha:", alpha)
    print("S:", S)
    print("C:", C)     

def handle_events():
    global active_field
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.MOUSEBUTTONDOWN:
            mouse_pos = pygame.mouse.get_pos()
            for key, value in parameters.items():
                rect = pygame.Rect(500, 30 + 40 * list(parameters.keys()).index(key), 100, 32)  # Reduced height to 40 pixels
                if rect.collidepoint(mouse_pos):
                    active_field = key
            if screen_width//2 - 50 < mouse_pos[0] < screen_width//2 + 50 and screen_height - 100 < mouse_pos[1] < screen_height - 50:
                flag = update_variables()
                print_variables()
                return flag
        if event.type == pygame.KEYDOWN:
            if active_field:
                if event.key == pygame.K_RETURN:
                    active_field = ""
                elif event.key == pygame.K_BACKSPACE:
                    parameters[active_field] = parameters[active_field][:-1]
                else:
                    parameters[active_field] += event.unicode

# Function to generate random points on a circle
def generate_points_on_circle(center, radius, num_points):
    points = []
    for _ in range(num_points):
        angle = random.uniform(0, 2 * math.pi)
        x = center[0] + radius * math.cos(angle)
        y = center[1] + radius * math.sin(angle)
        points.append((int(x), int(y)))
    return points

# Draw concentric circles and the square box at the center
def draw_environment():
    screen.fill((135, 206, 250))
    pygame.draw.circle(screen, (144, 238, 144), (screen_width // 2, screen_height // 2), R_det, 0)
    pygame.draw.circle(screen, (255, 195, 77), (screen_width // 2, screen_height // 2), R_c, 0)
    pygame.draw.circle(screen, (255, 160, 0), (screen_width // 2, screen_height // 2), R_min, 0)
    pygame.draw.rect(screen, RED, (screen_width // 2 - ( adsSize//2 ), screen_height // 2 - ( adsSize//2 ), adsSize, adsSize))

# Draw squares at the random points on the outermost circle
def draw_random_points(points, R):
    for point in points:
        pygame.draw.rect(screen, RED, (point[0] - (droneSize//2), point[1] - (droneSize//2), droneSize, droneSize))
        health_bar_x = point[0] - (health_bar_width//2)
        health_bar_y = point[1] - (droneSize//2) - health_bar_height
        
        pygame.draw.rect(screen, BLACK, (health_bar_x, health_bar_y, health_bar_width, health_bar_height))
        health_bar_width_remaining = int(health_bar_width * R[points.index(point)] / r)
        pygame.draw.rect(screen, GREEN, (health_bar_x, health_bar_y, health_bar_width_remaining, health_bar_height))
        
        
   
def update_points_position(points):
    center_x = screen_width // 2
    center_y = screen_height // 2
    for i in range(len(points)):
        point_x, point_y = points[i]
        distance = math.sqrt((center_x - point_x) ** 2 + (center_y - point_y) ** 2)
        if distance == 0:
            continue
        dx = (center_x - point_x) * v_d / distance
        dy = (center_y - point_y) * v_d / distance
        points[i] = (point_x + dx, point_y + dy)

    return points

def draw_assignment_lines(points, assignment):
    center_x = screen_width // 2
    center_y = screen_height // 2
    for i in range(len(assignment)):
        drone_index = assignment[i]
        if drone_index < len(points):
            pygame.draw.line(screen, RED, (center_x, center_y), points[drone_index], 2)

def f(num_strikes_till_now, param = 0.005, a = 999):
    return 1/(1+a*math.exp(-param*num_strikes_till_now))

def iteration(L, R, t0, m_f, assignment):
    L_new = []
    for num_lasers in L:
        num_failures = sum([1 if random.random() < f(t0) else 0 for _ in range(num_lasers)])
        L_new.append(num_lasers - num_failures)
    
    print(L_new)    
    countLasers = [0 for _ in range(len(R))]
    for i in range(len(assignment)):
        countLasers[assignment[i]] += L[i]
    R_new = []
    sum_failures = 0
    for i in range(len(R)):
        num_failures = sum([1 if random.random() < m_f else 0 for _ in range(countLasers[i])] )
        num_strikes = countLasers[i] - num_failures
        R_new.append(max(0, R[i] - num_strikes))
        sum_failures = sum_failures + num_failures
    
    print(sum_failures)    
    print(R_new)
    return [L_new, R_new]

def main_loop():
    # Initialize the environment    
    points_on_outer_circle = generate_points_on_circle((screen_width // 2, screen_height // 2), R_det, n_d)
    L = [l for _ in range(n_s)]
    R = [r for _ in range(n_d)]
    assignment = list(random.randint(0, len(R)-1) for _ in range(len(L)))
    
    # Main game loop
    running = True
    t = 0
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        draw_environment()
        draw_random_points(points_on_outer_circle, R)

        center_x, center_y = (screen_width // 2, screen_height // 2)
        distance = min(math.sqrt((center_x - point_x) ** 2 + (center_y - point_y) ** 2) for point_x, point_y in points_on_outer_circle)

        tq = t % (alpha*S + C)
        if ( tq <= alpha*S and distance <= R_c):
            draw_assignment_lines(points_on_outer_circle, assignment)
            if ( tq % S == 0 ):
                
                t0 = alpha*math.floor(t/(S*alpha + C)) + min(math.floor(t%(S*alpha + C)/S), alpha)
                [L, R] = iteration(L, R, t0, m_f, assignment)
                
                R_new = [index for index in range(len(R)) if R[index] > 0]
                if len(R_new) == 0:
                    running = False
                    continue
                assignment = list(random.choice(R_new) for _ in range(len(L)))
            

        new_points_on_outer_circle = []
        for i in range(len(R)):
            if ( R[i] > 0 ):
                new_points_on_outer_circle.append(points_on_outer_circle[i])
            else:
                new_points_on_outer_circle.append((2*screen_width, 2*screen_height))
        points_on_outer_circle = new_points_on_outer_circle
        points_on_outer_circle = update_points_position(points_on_outer_circle)
        distance = min(math.sqrt((center_x - point_x) ** 2 + (center_y - point_y) ** 2) for point_x, point_y in points_on_outer_circle)
        
        if ( distance <= R_c):
            t = t + 1

        if( distance <= R_min):
            running = False
        
        pygame.display.flip()
        pygame.time.Clock().tick(5)
        
    return len([r for r in R if r > 0]) == 0    

def welcome_screen():
    screen.fill(WHITE)
    font = pygame.font.Font(None, 36)
    text = font.render("Welcome to the simulation", True, BLACK)
    text_rect = text.get_rect(center=(screen_width // 2, screen_height // 2 - 50))
    screen.blit(text, text_rect)

    continue_button = pygame.Rect(screen_width // 2 - 100, screen_height // 2, 200, 50)
    pygame.draw.rect(screen, GREEN, continue_button)

    font = pygame.font.Font(None, 30)
    text = font.render("Continue", True, BLACK)
    text_rect = text.get_rect(center=continue_button.center)
    screen.blit(text, text_rect)

    pygame.display.flip()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if continue_button.collidepoint(event.pos):
                    return

def success_screen():
    screen.fill(WHITE)  # Set background to white
    
    # Create a green rectangle
    success_rect = pygame.Rect(screen_width // 3, screen_height // 5, screen_width // 2, screen_height // 2)
    pygame.draw.rect(screen, GREEN, success_rect)
    
    font = pygame.font.Font(None, 36)
    text = font.render("Objective completed successfully", True, BLACK)
    text_rect = text.get_rect(center=success_rect.center)
    screen.blit(text, text_rect)

    pygame.display.flip()
    pygame.time.delay(1000)
    pygame.quit()
    sys.exit()


def failure_screen():
    screen.fill(WHITE)
    # Create a green rectangle
    fail_rect = pygame.Rect(screen_width // 3, screen_height // 5, screen_width // 2, screen_height // 2)
    pygame.draw.rect(screen, RED, fail_rect)
    
    font = pygame.font.Font(None, 36)
    text = font.render("Objective failed", True, BLACK)
    text_rect = text.get_rect(center=fail_rect.center)
    screen.blit(text, text_rect)

    pygame.display.flip()
    pygame.time.delay(1000)
    pygame.quit()
    sys.exit()
    
# Main function
def main():
    welcome_screen()
    run = True
    while run:
        run = not handle_events()
        draw_ui()
        pygame.display.flip()
    final_result = main_loop()
    if final_result:
        success_screen()
    else:
        failure_screen()    

if __name__ == "__main__":
    main()



n_s: 2
n_d: 5
v_d: 3.0
R_min: 10
R_c: 200
R_det: 300
r: 3
l: 2
m_f: 0.25
alpha: 6
S: 10
C: 240
[2, 2]
1
[3, 1, 3, 3, 2]
[2, 2]
2
[3, 0, 3, 3, 2]
[2, 2]
0
[3, 0, 3, 3, 0]
[2, 2]
1
[3, 0, 0, 3, 0]
[2, 2]
2
[3, 0, 0, 1, 0]
[2, 2]
2
[2, 0, 0, 0, 0]


SystemExit: 